I am using Python 3.9.0.  To begin:

```
pip install backtrader
pip install backtrader_plotting
pip install yfinance

```




In [59]:
# %matplotlib inline

from datetime import datetime
import backtrader as bt
import yfinance as yf
from backtrader_plotting import Bokeh
import pandas as pd

# open HMM states file
with open('states_df.csv', 'r') as f:
    states_df = pd.read_csv(f)

#set 'Date' column to datetime.date
states_df['Date'] = pd.to_datetime(states_df['Date'])

# search for today and return the state
def get_state(date, df):
    # search df for date
    # return state
    return df[df['Date'] == date]['State'].values[0]


# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=200   # period for the slow moving average
    )
    
    

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.params.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.params.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):

        self.today = self.datas[0].datetime.date(0)     # get todays date, THIS IS CORRECT


        if self.today in states_df['Date'].values:                  # there is a problem here
            self.state = get_state(self.today, states_df)           # there is a problem here

        else: self.state = 0

        if not self.position:  # not in the market
            if self.crossover > 0 and self.state == 0: # why does this not work?
                self.order_target_percent(target=1)
                self.buy() # enter long 100% of equity

        elif self.crossover < 0:  # in the market & cross to the downside
            self.order_target_percent(target=0)  # close long position




cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Backtest date range
startdate = datetime(2007, 1, 1)
enddate = datetime(2018, 1, 1)

# Choose a ticker
ticker = 'SPX'

# Create a data feed
data = bt.feeds.YahooFinanceCSVData(dataname=f'data/{ticker}.csv',
                                    fromdate=startdate,
                                    adjclose=False,
                                    adjvolume=False,
                                    todate=enddate)
                                    
cerebro.adddata(data)  # Add the data feed
cerebro.broker.setcash(100000)  # Set our starting cash
cerebro.addstrategy(SmaCross)  # Add the trading strategy
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()  # run it all
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
b = Bokeh(style='bar', plot_mode='single',)
cerebro.plot(b)  # and plot it with a single command



Starting Portfolio Value: 100000.00
Final Portfolio Value: 216933.71


[[<backtrader_plotting.bokeh.bokeh.FigurePage at 0x7fbc4f54c280>]]

pandas.core.series.Series